## Посчитать a^{1/k}

In [39]:
def root_binsection(a, k):
    l = 0
    r = a + 1
    for i in range(100):
        m = (l + r) / 2
        if m ** k >= a:
            r = m
        else:
            l = m
    return l

print(root_binsection(125, 3))
print(root_binsection(0.125, 3))

4.999999999999999
0.49999999999999994


In [40]:
def root_newton(a, k):
    x = a + 1
    for i in range(100):
        x = x - (x**k - a) / (k * x**(k - 1))
    return x

print(root_newton(125, 3))
print(root_newton(0.125, 3))

5.0
0.5


## Про многочлен 

У многочлен имеет вид x**i * p[i]

In [41]:
eps = 1e-7

def derivative(poly):
    return [(i + 1) * c for i, c in enumerate(poly[1:])]

def at(x, poly):
    res = 0
    for i, c in enumerate(poly):
        res += c * x**i
    return res

def sign(val):
    return 1 if val > 0 else -1

def normalize(poly):
    while len(poly) > 0 and poly[-1] == 0:
        poly.pop()
    return poly

def zero(val):
    return abs(val) < eps

print(derivative([1, 2, 3]))
print(at(2, [1, 2, 3]))
print(sign(-7), sign(7))
print(len(normalize([1, 2, 0, 0, 0])), len(normalize([0, 0, 0])))

[2, 6]
17
-1 1
2 0


In [42]:
def roots(poly, l, r):
    poly = normalize(poly)
    assert(len(poly) > 0)
    if len(poly) == 1:
        return []
    if len(poly) == 2:
        return [-poly[0] / poly[1]]
    res = []
    for l, r in localize(poly, l, r):
        for i in range(100):
            m = (l + r) / 2
            if sign(at(m, poly)) == sign(at(l, poly)):
                l = m
            else:
                r = m
        res.append(l)
    return res
    

def localize(poly, l, r):
    poly = normalize(poly)
    assert(len(poly) > 0)
    if zero(at(l, poly)):
        l -= eps
    if zero(at(r, poly)):
        l += eps
    deri_roots = roots(derivative(poly), l, r) + [r]
    s = [l]
    for x in deri_roots:
        if sign(at(s[-1], poly)) != sign(at(x, poly)):
            s.append(x)
    return [[s[i], s[i + 1]] for i in range(len(s) - 1)]

print(localize([-1, 0, 1], -2, 2)) # x^2 - 1, L = -2, R = 2
print(roots([-1, 0, 1], -2, 2)) # x^2 - 1, L = -2, R = 2

[[-2, 0.0], [0.0, 2]]
[-1.0000000000000002, 1.0]


In [43]:
def get_min(poly, l, r):
    res = min(at(l, poly), at(r, poly))
    for r in roots(derivative(poly), l, r):
        res = min(res, at(r, poly))
    return res

print(get_min([1, 0, 1], -2, 2))
print(get_min([-1, 2, 1], -2, 2))

1.0
-2.0


## Найти минимум e^{ax} + e^{-bx} + c(x - d)^2

In [56]:
# from math import exp
from random import uniform

def f(x, a, b, c, d):
    return exp(a * x) + exp(-b * x) + c * (x - d)**2

def df(x, a, b, c, d):
    return a * exp(a * x) - b * exp(-b * x) + 2 * c * (x - d)

def ddf(x, a, b, c, d):
    return a**2 * exp(a * x) + b**2 * exp(-b * x) + 2 * c


def true_min(a, b, c, d, l = -20, r = 20, part = 10**5):
    res = f(0, a, b, c, d)
    for i in range(part):
        x = (r - l) / part * i + l
        res = min(res, f(x, a, b, c, d))
    return res

def min_bisection(a, b, c, d):
    bound = 1
    mini = f(d, a, b, c, d)
    while exp(bound * a) < mini or exp(bound * b) < mini:
        bound *= 2
    l = -bound
    r = bound
    for i in range(100):
        m = (l + r) / 2
        if df(m, a, b, c, d) < 0:
            l = m
        else:
            r = m
    return f(l, a, b, c, d)

def min_newton(a, b, c, d):
    x = d
    for i in range(100):
        x = x - df(x, a, b, c, d) / ddf(x, a, b, c, d)
    return f(x, a, b, c, d)

def min_ternary(a, b, c, d):
    bound = 1
    mini = f(d, a, b, c, d)
    while exp(bound * a) < mini or exp(bound * b) < mini:
        bound *= 2
    l = -bound
    r = bound
    for i in range(100):
        m1 = l + (r - l) / 3
        m2 = r - (r - l) / 3
        if f(m1, a, b, c, d) < f(m2, a, b, c, d):
            r = m2
        else:
            l = m1
    return f(l, a, b, c, d)

for i in range(5):
    a = uniform(0, 5)
    b = uniform(0, 5)
    c = uniform(0, 5)
    d = uniform(0, 10)
    real_min = true_min(a, b, c, d)
    min_bis = min_bisection(a, b, c, d)
    min_new = min_newton(a, b, c, d)
    min_ter = min_ternary(a, b, c, d)
    assert(abs(real_min - min_bis) < 1e-2)
    assert(abs(real_min - min_new) < 1e-2)
    assert(abs(real_min - min_ter) < 1e-2)
    